<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/gliner/gliner2_blog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet --upgrade gliner2 neo4j

In [3]:
from gliner2 import GLiNER2

# Load model once, use everywhere
extractor = GLiNER2.from_pretrained("fastino/gliner2-large-v1")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

🧠 Model Configuration
Encoder model      : microsoft/deberta-v3-large
Counting layer     : count_lstm
Token pooling      : first


model.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

In [4]:
text = """Augusta Ada King, Countess of Lovelace (10 December 1815 – 27 November 1852), also known as Ada Lovelace, was an English mathematician and writer chiefly known for work on Charles Babbage's proposed mechanical general-purpose computer, the analytical engine. She was the first to recognise the machine had applications beyond pure calculation. Lovelace is often considered the first computer programmer.

Lovelace was the only legitimate child of poet Lord Byron and reformer Anne Isabella Milbanke. All her half-siblings, Lord Byron's other children, were born out of wedlock to other women. Lord Byron separated from his wife a month after Ada was born, and left England forever. He died in Greece during the Greek War of Independence, when she was eight. Lady Byron was anxious about her daughter's upbringing and promoted Lovelace's interest in mathematics and logic, to prevent her developing her father's perceived insanity. Despite this, Lovelace remained interested in her father, naming one son Byron and the other, for her father's middle name, Gordon. Lovelace was buried next to her father at her request.

Although often ill in childhood, Lovelace pursued her studies assiduously. She married William King in 1835. King was a Baron, and was created Viscount Ockham and 1st Earl of Lovelace in 1838. The name Lovelace was chosen because Ada was descended from the extinct Baron Lovelaces.[4] The title given to her husband thus made Ada the Countess of Lovelace."""

In [5]:
entities = extractor.extract_entities(
    text,
    {
        "Person": "Names of people, including nobility titles.",
        "Location": "Countries, cities, or geographic places.",
        "Invention": "Machines, devices, or technological creations.",
        "Event": "Historical events, wars, or conflicts."
    }
)
entities

{'entities': {'Person': ['Ada Lovelace',
   'Anne Isabella Milbanke',
   'Lord Byron',
   'William King',
   'Charles Babbage',
   'Augusta Ada King',
   'Lady Byron',
   'Viscount Ockham',
   'Byron',
   'Countess of Lovelace',
   '1st Earl of Lovelace',
   'Gordon'],
  'Location': ['Greece', 'England'],
  'Invention': ['analytical engine'],
  'Event': ['Greek War of Independence']}}

In [6]:
relations = extractor.extract_relations(
    text,
    {
        "parent_of": "A person is the parent of another person",
        "married_to": "A person is married to another person",
        "worked_on": "A person contributed to or worked on an invention",
        "invented": "A person created or proposed an invention",
        "alias": "Entity is an alias, nickname, title, or alternate reference for another entity",
        "same_as": "Entity is an alias, nickname, title, or alternate reference for another entity"
    }
)
relations

{'relation_extraction': {'parent_of': [('Lord Byron', 'Ada Lovelace'),
   ('Anne Isabella Milbanke', 'Ada Lovelace')],
  'married_to': [('Ada Lovelace', 'William King')],
  'worked_on': [('Augusta Ada King', 'analytical engine')],
  'invented': [('Charles Babbage', 'analytical engine')],
  'alias': [('Augusta Ada King', 'Ada Lovelace')],
  'same_as': []}}

In [7]:
schema = (extractor.create_schema()
    .entities({
        "Person": "Names of people, including nobility titles.",
        "Location": "Countries, cities, or geographic places.",
        "Invention": "Machines, devices, or technological creations.",
        "Event": "Historical events, wars, or conflicts."
    })
    .relations({
        "parent_of": "A person is the parent of another person",
        "married_to": "A person is married to another person",
        "worked_on": "A person contributed to or worked on an invention",
        "invented": "A person created or proposed an invention",
        "alias": "Entity is an alias, nickname, title, or alternate reference for another entity"
    })
)

results = extractor.extract(text, schema)
results

{'entities': {'Person': ['William King',
   'Lord Byron',
   'Anne Isabella Milbanke',
   'Ada Lovelace',
   'Augusta Ada King',
   'Charles Babbage',
   'Lady Byron',
   'Byron'],
  'Location': ['Greece', 'England'],
  'Invention': ['analytical engine'],
  'Event': ['Greek War of Independence']},
 'relation_extraction': {'parent_of': [('Lord Byron', 'Ada Lovelace'),
   ('Anne Isabella Milbanke', 'Ada Lovelace')],
  'married_to': [('Ada Lovelace', 'William King')],
  'worked_on': [('Augusta Ada King', 'analytical engine')],
  'invented': [('Charles Babbage', 'analytical engine')],
  'alias': [('Augusta Ada King', 'Ada Lovelace')]}}

In [8]:
schema = (extractor.create_schema()
    .entities({
        #"Person": "Names of people, including nobility titles.",
        #"Location": "Countries, cities, or geographic places.",
        "Invention": "Machines, devices, or technological creations.",
        "Event": "Historical events, wars, or conflicts."
    })
    .relations({
        "parent_of": "A person is the parent of another person",
        "married_to": "A person is married to another person",
        "worked_on": "A person contributed to or worked on an invention",
        "invented": "A person created or proposed an invention",
        "alias": "Entity is an alias, nickname, title, or alternate reference for another entity"
    })
)

results = extractor.extract(text, schema)
results

{'entities': {'Invention': ['analytical engine'],
  'Event': ['Greek War of Independence']},
 'relation_extraction': {'parent_of': [('Lord Byron', 'Ada Lovelace'),
   ('Anne Isabella Milbanke', 'Ada Lovelace')],
  'married_to': [('Ada Lovelace', 'William King')],
  'worked_on': [('Augusta Ada King', 'analytical engine')],
  'invented': [('Charles Babbage', 'analytical engine')],
  'alias': [('Augusta Ada King', 'Ada Lovelace')]}}

In [9]:
results = extractor.extract_json(
    text,
    {
        "person": [
            "name::str",
            "gender::str::male or female",
            "alias::str",
            "description::str",
            "birth_date::str",
            "death_date::str",
            "parent_of::str",
            "married_to::str"
        ]
    }
)
results

{'person': [{'name': 'Augusta Ada King',
   'gender': None,
   'alias': 'Ada Lovelace',
   'description': 'English mathematician and writer',
   'birth_date': '10 December 1815',
   'death_date': '27 November 1852',
   'parent_of': 'Ada Lovelace',
   'married_to': 'William King'},
  {'name': 'Charles Babbage',
   'gender': None,
   'alias': None,
   'description': None,
   'birth_date': None,
   'death_date': None,
   'parent_of': 'Ada Lovelace',
   'married_to': None},
  {'name': 'Lord Byron',
   'gender': None,
   'alias': None,
   'description': 'reformer',
   'birth_date': None,
   'death_date': None,
   'parent_of': 'Ada Lovelace',
   'married_to': None},
  {'name': 'Anne Isabella Milbanke',
   'gender': None,
   'alias': None,
   'description': 'reformer',
   'birth_date': None,
   'death_date': None,
   'parent_of': 'Ada Lovelace',
   'married_to': None},
  {'name': 'William King',
   'gender': None,
   'alias': None,
   'description': None,
   'birth_date': None,
   'death_date

In [10]:
schema = (extractor.create_schema()
    .entities({
        "Person": "Names of people, including nobility titles.",
        "Location": "Countries, cities, or geographic places.",
        "Invention": "Machines, devices, or technological creations.",
        "Event": "Historical events, wars, or conflicts."
    })
    .relations({
        "parent_of": "A person is the parent of another person",
        "married_to": "A person is married to another person",
        "worked_on": "A person contributed to or worked on an invention",
        "invented": "A person created or proposed an invention",
    })
    .structure("person")
        .field("name", dtype="str")
        .field("alias", dtype="str")
        .field("description", dtype="str")
        .field("birth_date", dtype="str")
)

results = extractor.extract(text, schema)
results

{'person': [{'name': 'Augusta Ada King',
   'alias': 'Ada Lovelace',
   'description': 'English mathematician and writer',
   'birth_date': '10 December 1815'},
  {'name': 'Lord Byron',
   'alias': None,
   'description': 'poet',
   'birth_date': None},
  {'name': 'Anne Isabella Milbanke',
   'alias': None,
   'description': 'reformer',
   'birth_date': None}],
 'entities': {'Person': ['William King',
   'Lord Byron',
   'Augusta Ada King',
   'Anne Isabella Milbanke',
   'Charles Babbage',
   'Ada Lovelace',
   'Lady Byron',
   'Byron'],
  'Location': ['Greece', 'England'],
  'Invention': ['analytical engine'],
  'Event': ['Greek War of Independence']},
 'relation_extraction': {'parent_of': [('Lord Byron', 'Ada Lovelace'),
   ('Lord Byron', 'Ada Lovelace')],
  'married_to': [('Ada Lovelace', 'William King')],
  'worked_on': [('Ada Lovelace', 'analytical engine')],
  'invented': [('Charles Babbage', 'analytical engine')]}}

In [11]:
from neo4j import AsyncGraphDatabase

NEO4J_URI = "bolt://3.227.9.233:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "wear-sheets-dividers"

driver = AsyncGraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [12]:
import_cypher_query = """
// Create Chunk node from text
CREATE (c:Chunk {text: $text})

// Create Person nodes with properties
WITH c
CALL (c) {
  UNWIND $data.person AS p
  WITH p
  WHERE p.name IS NOT NULL
  MERGE (n:__Entity__ {name: p.name})
  SET n.description = p.description,
      n.birth_date = p.birth_date
  MERGE (c)-[:MENTIONS]->(n)
  WITH p, n WHERE p.alias IS NOT NULL
  MERGE (m:__Entity__ {name: p.alias})
  MERGE (n)-[:ALIAS_OF]->(m)
}

// Create entity nodes dynamically with __Entity__ base label + dynamic label
CALL (c) {
  UNWIND keys($data.entities) AS label
  UNWIND $data.entities[label] AS entityName
  MERGE (n:__Entity__ {name: entityName})
  SET n:$(label)
  MERGE (c)-[:MENTIONS]->(n)
}

// Create relationships dynamically
CALL (c) {
  UNWIND keys($data.relation_extraction) AS relType
  UNWIND $data.relation_extraction[relType] AS rel
  MATCH (a:__Entity__ {name: rel[0]})
  MATCH (b:__Entity__ {name: rel[1]})
  MERGE (a)-[:$(toUpper(relType))]->(b)
}
RETURN distinct 'import completed' AS result
"""


In [13]:
await driver.execute_query(import_cypher_query, data=results, text=text)

EagerResult(records=[<Record result='import completed'>], summary=<neo4j._work.summary.ResultSummary object at 0x77feae7808f0>, keys=['result'])